In [65]:
from google.cloud import aiplatform

import tensorflow as tf
import numpy as np
from PIL import Image
import glob
import time

In [2]:
# Download image labels

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

10484/10484 [==============================] - 0s 0us/step


In [3]:
PROJECT_NUMBER = "428711328705"
ENDPOINT_ID = "2493098635516968960"

endpoint = aiplatform.Endpoint(
    endpoint_name=f"projects/{PROJECT_NUMBER}/locations/us-central1/endpoints/{ENDPOINT_ID}")

In [299]:
file = 'Imagenet/*.JPEG'
file_string = glob.glob(file)
IMAGE_SIZE = (200, 200)
x_instances = []
njobs = 100 # number of jobs
offset = 282

for x in range(njobs):
    # print(file_string[x+offset])
    im = Image.open(file_string[x+offset])
    im = im.resize(IMAGE_SIZE)
    im = np.array(im)/255.0
    x_test = im.astype(np.float32).tolist()
    x_instances.append(x_test)

In [ ]:
file = 'Imagenet/*.JPEG'
file_string = glob.glob(file)
IMAGE_SIZE = (200, 200)
x_instances = []
njobs = 400 # number of jobs
offset = 100

for x in range(njobs):
    # print(file_string[x+offset])
    im = Image.open(file_string[x+offset])
    im = im.resize(IMAGE_SIZE)
    im = np.array(im)/255.0
    x_test = im.astype(np.float32).tolist()
    x_instances.append(x_test)

#generating inter arrival times
inter_arrival_times = [0.1 for i in range(njobs)]
# inter_arrival_times = [0.4 for i in range(5)] + [10] + [0.4 for i in range(4)] # high low inter arrival

arrival_times= []# list of arrival times of a person joining the queue
service_times = [] # list of service times once they reach the front
finish_times = [] # list of finish times after waiting and being served
    
arrival_times = [0 for i in range(njobs)]
service_times = [0 for i in range(njobs)]
finish_times = [0 for i in range(njobs)]
    
arrival_times[0]=round(inter_arrival_times[0],2)#arrival of first customer
#Generate arrival times
for i in range(1,njobs):
    arrival_times[i]=round((arrival_times[i-1]+inter_arrival_times[i]),2)
    
startTime = currentTime = finishTime = time.perf_counter()
curCust = 0

while (curCust < njobs):
    currentTime = time.perf_counter()
    if(currentTime - startTime >= arrival_times[curCust]):
        print(file_string[curCust+offset])
        result = endpoint.predict(instances=x_instances[curCust:(curCust+1)]).predictions
        finishTime = time.perf_counter()
        finish_times[curCust] = finishTime - startTime
        service_times[curCust] = round(finishTime - currentTime, 2)
        curCust += 1
        
print("total run time", (finishTime - startTime))

# Total time spent in the system by each customer
total_times =[abs(round((finish_times[i]-arrival_times[i]),2)) for i in range(njobs)]

# Time spent@waiting before being served (time spent in the queue)
wait_times = [abs(round((total_times[i] - service_times[i]),2)) for i in range(njobs)]

print('arrival times', arrival_times)
print('response time', total_times)
print('service time', service_times)
print('wait time', wait_times)

Imagenet/n02111129_Leonberg.JPEG
Imagenet/n03290653_entertainment_center.JPEG
Imagenet/n09399592_promontory.JPEG
Imagenet/n03447447_gondola.JPEG
Imagenet/n03496892_harvester.JPEG
Imagenet/n02783161_ballpoint.JPEG
Imagenet/n03956157_planetarium.JPEG
Imagenet/n03127925_crate.JPEG
Imagenet/n02088632_bluetick.JPEG
Imagenet/n02110958_pug.JPEG
Imagenet/n04591157_Windsor_tie.JPEG
Imagenet/n03584829_iron.JPEG
Imagenet/n02002724_black_stork.JPEG
Imagenet/n03272010_electric_guitar.JPEG
Imagenet/n02123159_tiger_cat.JPEG
Imagenet/n04039381_racket.JPEG
Imagenet/n03075370_combination_lock.JPEG
Imagenet/n04026417_purse.JPEG
Imagenet/n03825788_nipple.JPEG
Imagenet/n07831146_carbonara.JPEG
Imagenet/n04209239_shower_curtain.JPEG
Imagenet/n02441942_weasel.JPEG
Imagenet/n03534580_hoopskirt.JPEG
Imagenet/n04509417_unicycle.JPEG
Imagenet/n02971356_carton.JPEG
Imagenet/n02093859_Kerry_blue_terrier.JPEG
Imagenet/n09468604_valley.JPEG
Imagenet/n02437312_Arabian_camel.JPEG
Imagenet/n03063689_coffeepot.JPEG
Imag